Goal of this notebook to test sms spam detection with valious classfiers

### Start

First of all neccesary imports

In [ ]:
import numpy as np
import pandas as pd
import string
from scipy import stats
import itertools

import mlflow as mf # install mlflow

# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# preprocessing
from nltk.stem import SnowballStemmer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords

# tokenizing
import tensorflow_hub as hub
import urllib.request
from tokenizers import BertWordPieceTokenizer
from nltk.tokenize import word_tokenize 

#model
import lightgbm as lgbm



Read the data from csv file

In [ ]:
sms = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding = 'Latin-1')
sms.head()

Now drop "unnamed" columns and rename v1 and v2 to "label" and "message"

In [ ]:
sms = sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
sms = sms.rename(columns = {'v1':'label','v2':'message'})

Check the changed data

In [ ]:
sms.groupby('label').describe()

Create new feature "message length" and plot it to see if it's of any interest

In [ ]:
sms['length'] = sms['message'].apply(len)
sms.head()

In [ ]:
mpl.rcParams['patch.force_edgecolor'] = True
plt.style.use('seaborn-bright')
sms.hist(column='length', by='label', bins=50,figsize=(11,5))

Looks like the lengthy is the message, more likely it is a spam. Let's not forget this

### Outlier analysis: Approaches such as box-plot can be used to detect outliers and deal with them

In [ ]:
def outliers_iqr(data):
    
    print(data.sample(10))
    green_diamond = dict(markerfacecolor='g', marker='D')
    plt.figure(figsize=(12,8))
    plt.boxplot(data['length'], flierprops=green_diamond)
    plt.show
    
    q1, q3 = np.percentile(data['length'],[25,75]) # 넘파이의 값을 퍼센트로 표시
    iqr = q3 -q1
    lower_bound = q1 - (iqr*1.5)
    upper_bound = q3 + (iqr*1.5)
    
    outlier_idx = data['length'][ (data['length'] < lower_bound) | (data['length'] > upper_bound) ].index
    
    return outlier_idx

In [ ]:
outliers_1 = outliers_iqr(sms[sms['label']=='spam'])
outliers_2 = outliers_iqr(sms[sms['label']=='ham'])

In [ ]:
#outliters = list(outliters)
print(outliers_1)
print(outliers_2)

In [ ]:
print(sms.head(50))

In [ ]:
sms.drop(index=outliers_1, axis=0, inplace=True)
sms.drop(index=outliers_2, axis=0, inplace=True)

In [ ]:
print(sms.head(50))

### Text preprocessing

1. Lower casing (소문자화)
2. Text cleansing(remove) like . , ! $( ) * % @
3. Removing Stop words (불용어 제거)
4. Stemming (형태소분석)
5. Lemmatization(표제어) -- 필요 없을듯

**#1. Lower preprocessing**

In [ ]:
sms['message'] = sms['message'].apply(lambda x: x.lower())

**#2. Text Cleansing**

In [ ]:
sms['message'] = sms['message'].str.replace('\(?(http|https|ftp|ftps)?\:\/\/[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(\/\S*)?\)?','')
sms['message'] = sms['message'].str.replace('\d{2,3}-?\d{3,4}-?\d{4}','')
sms['message'] = sms['message'].str.replace('([\w\.-]+)@([\w\.-]+)(\.[\w\.]+)','')

**#3. Stop Words & Stemmer**

In [ ]:
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer("english")

In [ ]:
def preprocess(text, stem=False):

    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [ ]:
sms['cleansing_text'] = sms['message']

In [ ]:
%%time
sms['cleansing_text'] = sms['cleansing_text'].apply(lambda x: preprocess(x))

after preprocessing...

In [ ]:
sms.head()

### Data label convert to string to float

In [ ]:
sms.loc[(sms['label'] == "ham"), 'label'] = 0  # ham => 0
sms.loc[(sms['label'] == "spam"), 'label'] = 1  # spam => 1
sms['label'] = sms['label'].astype('float32')

###  Classifiers and predictions

First of all let's split our features to test and train set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sms['cleansing_text'] ,sms['label'], test_size=0.2, random_state=111, shuffle=True)

## Model test 1. LightGBM

In [ ]:
# lightGBM import
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 

### bag of words

In [ ]:
tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)
X_data_tfidf = tfidf_vec.fit_transform(sms['cleansing_text'])
X_train_tfidf = tfidf_vec.transform(X_train)
X_test_tfidf = tfidf_vec.transform(X_test)

In [ ]:
print(X_train_tfidf)

In [ ]:
print(y_train)

### train and evaluate model

In [ ]:
# LGBM 분류기 객체 생성
lgbm_wrapper = LGBMClassifier(n_estimators=400)

# 조기 중단 기능에 필요한 파라미터 정의
evals = [(X_test_tfidf, y_test)]
lgbm_wrapper.fit(X_train_tfidf, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)

In [ ]:
# model test
y_pred = lgbm_wrapper.predict(X_test_tfidf)

### test graph visualization

In [ ]:
#confusion matrix 그래프 설정
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12,12))
plot_confusion_matrix(cnf_matrix, classes= ['ham','spam'], title="Confusion matrix")
plt.show()

In [ ]:
print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))